In [1]:
from sweep_lib import SorceMeater, VoltCraft
import time
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [2]:
results_combined = pd.DataFrame()
### Define some funktions
def sweep(VC,SM,tpl,setpoints,setpoints_V,pol_changed):
    results = pd.DataFrame()
    # measure 0V (only measured with the pos polarity)
    if not pol_changed:
        VC.disable_sorce()
        time.sleep(1)
        SM.initialise()
        time.sleep(tpl)
        data = SM.read_buffer()
        data['bias'] = 0
        data['abs_Read'] = abs(data['Read'])
        results = pd.concat([results, data], ignore_index=True, sort=False)
    
    for i in range(len(setpoints)):
        VC.set_voltage(setpoints[i])
        VC.enable_sorce()
        time.sleep(1)
        SM.initialise()
        time.sleep(tpl)
        data = SM.read_buffer()
        data['abs_Read'] = abs(data['Read'])
        if pol_changed:
            data['bias'] = -setpoints_V[i]
        else:
            data['bias'] = setpoints_V[i]
        results = pd.concat([results, data], ignore_index=True, sort=False)
    VC.set_voltage(0)
    VC.disable_sorce()
    return results

def pos_sweep(VC,SM,tpl,setpoints,setpoints_V):
    return sweep(VC,SM,tpl,setpoints,setpoints_V,False)

def neg_sweep(VC,SM,tpl,setpoints,setpoints_V):
    return sweep(VC,SM,tpl,setpoints,setpoints_V,True)

def plot(results, sample_folder,sample_name,log_scale:bool = False,write_html:bool=True):
    list_V = [0.1,0,-0.1]
    a = 0.001
    fig = go.Figure()
    for i in list_V:
        data =results[results['Source']>=i-a][results['Source']<=i+a]
        fig.add_trace(go.Scatter(x=data['bias'],y=data['abs_Read'],mode='markers', name = 'V(DS) = '+str(i) + ' V'))

    fig.update_layout(
        xaxis_title="V(GS)",
        yaxis_title="I(DS)",
    )
    if log_scale:
        fig.update_yaxes(type="log")
    
    if write_html:
        fig.write_html(sample_folder + sample_name +'_Bias_vs_Current_lin.html')
    return fig

In [4]:
# Setup Voltcraft power supply
vc = VoltCraft('COM6')
vc.disable_sorce()
vc.set_voltage(0)

In [7]:
## Define Parameters
time_per_loop = 1 # time it takes for the keithley to perfom a full loop
sample_folder = 'Data\\SiO2_90nm\\221213\\' #path to save the data 
# Keithley
v_start = -0.1  # Start Voltage for Keithley in Volt
v_stop = 0.1    # Start Voltage for Keithley in Volt
points = 3      # Number of points to measure (keithley)
i_limit = 0.1   # current limit used by the keithley in Amps
delay = 0.1     # time waited by the Keithley bevore Sampeling
# Voltcraft
setpoints = [0,0.5,1,1.5,2,2.5,3,3.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9,9.5,10]        # Voltages set by the Voltcraft in Volt
setpoints_V = [0.288,0.737,1.203,1.683,2.17,2.67,3.18,3.68,4.18,4.68,5.18,5.68,6.19,6.7,7.2,7.7,8.20,8.69,9.19,9.69,10.19]# real outputtvoltages by the Voltcraft in Volt 
polarity_changed = False    # polarity of the Voltcraft (True == nomal, False == reverse=negativ voltages)

In [8]:
sm = SorceMeater()
sm.use_front_terimals()
sm.sens_current()
sm.set_sens_range('auto')
sm.sorce_voltage()
sm.set_sorce_range(max(v_start,v_stop))
sm.set_sorce_limit(i_limit)
sm.set_wire(two_wire=True)
sm.setup_voltage_sweep(v_start,v_stop,points,delay=delay)

In [ ]:
sm.initialise() # testfunktion for time_per_loop parameter

In [34]:
sample_name = '6'
results_pos = pos_sweep(vc,sm,time_per_loop,setpoints=setpoints,setpoints_V=setpoints_V)

In [35]:
results_neg = neg_sweep(vc,sm,time_per_loop,setpoints=setpoints,setpoints_V=setpoints_V)
results = pd.concat([results_neg, results_pos], ignore_index=True, sort=False)
fig = plot(results=results,sample_folder=sample_folder,sample_name=sample_name)
fig.show()
results.to_csv(sample_folder + sample_name +'.csv')
results['Name'] = sample_name
results_combined = pd.concat([results_combined, results], ignore_index=True, sort=False)

C:\Users\Daniel Matulka\AppData\Local\Temp\ipykernel_11692\2792620845.py:44: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



In [38]:
i = -0.1
a = 0.001
fig = go.Figure()
b = results_combined['Name'].unique()
b = b[1:]
for c in b:
    results = results_combined[results_combined['Name']==c]
    data =results[results['Source']>=i-a][results['Source']<=i+a]
    fig.add_trace(go.Scatter(x=data['bias'],y=data['abs_Read'],mode='markers', name = 'Sample: '+str(c)))

fig.update_layout(
    title="V(DS) = " +  str(i) + ' V',
    xaxis_title="V(GS)",
    yaxis_title="I(DS)"
)    
fig.show()
fig.write_html(sample_folder +str(i) + '_Overview_Bias_vs_Current_lin.html')
results_combined.to_csv(sample_folder + 'Overview_Data.csv')